In [10]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [9]:
# define the LSTM modelt
import time
starttime = time.time()

max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

model = Sequential()
model.add(SimpleRNN(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.add(Dense(len(y), activation='softmax'))

print('Build model...')
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)
endtime = time.time()

NameError: name 'X' is not defined